In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# Assuming 'args.device' is set to 'cuda' if a GPU is available, else 'cpu'
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Load the pre-trained RoBERTa model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model = model.to(device)

# Load the tokenizer
senti_tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Sample text
text = "User: I like to watch scary movies"

# Tokenize the text
inputs = senti_tokenizer(text, return_tensors="pt")#, padding=True, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Get the logits from the model
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()

# Note: twitter-roberta-senti model => [negative, neutral, positive]
negative, neutral, positive = probabilities[0]
sentiment_score = positive / (positive + negative)

print(sentiment_score)

In [ ]:
class TestLoop():
    def __init__(self, args, test_dataloader, save_name, epoch_num):
      self.device = args.device
      self.save_name = save_name
      self.epoch_num = epoch_num 
    
      self.test_dataloader = test_dataloader
      
      self.rec_model = RecModel(args.kg_emb_dim, n_entity=kg['num_entities'], num_relations=kg['num_relations'], 
                    num_bases=args.num_bases, edge_index=kg['edge_index'], edge_type=kg['edge_type'],
                    device=self.device)

      self.rec_model.to(self.device)
      
      self.rec_evaluator = RecEvaluator()
      
      self.best_metric_dir = os.path.join(args.output_dir, 'best')
    
    def load_model(self):
        self.rec_model.load_state_dict(torch.load(self.best_metric_dir + f'epoch_{self.epoch_num}_{self.save_name}.pth'))
        
    def test(self):
        logger.info('>>>>>>>>>>>>>>>>> Test Evaluation')
        self.load_model()
        self.rec_model.eval()
        
        test_loss = []
        for step, batch in enumerate(tqdm(self.test_dataloader)):
            with torch.no_grad():
                rec_score, rec_loss = self.rec_model(batch)
                test_loss.append(rec_loss)
                
                score = rec_score[:, kg['item_ids']]
                ranks = torch.topk(score, k=50, dim=-1).indices.tolist()
                ranks = [[kg['item_ids'][rank] for rank in batch_rank] for batch_rank in ranks] # 각 배치에 대한 상위 50개 엔티티의 인덱스 리스트
                labels = batch['context']['rec_labels']
                self.rec_evaluator.evaluate(ranks, labels)
                
        # metric
        metric_report = self.rec_evaluator.report()
        for k, v in metric_report.items():
            metric_report[k] = v.sum().item()
            
        test_report = {}
        for k, v in metric_report.items():
            if k != 'count':
                test_report[f'test/{k}'] = v / metric_report['count']
        
        test_report['valid/loss'] = np.mean(test_loss)
        # test_report['epoch'] = epoch
        
        logger.info(f'{test_report}')
        logger.info('end of test')
        
        self.rec_evaluator.reset_metric()

In [1]:
movieMentions= {"173191": "House at the End of the Street (2012)", "205430": "A Quiet Place (2018)", "118338": "The Forest  (2016)", "177915": "Uncle Buck", "132562": "The Last House on the Left  (1972)", "76279": "Death at a Funeral  (2007)", "158950": "The Last House on the Left  (2009)", "144779": "Annabelle 2 (2017)"}

seekerPolarity= {"173191": {"suggested": 1, "seen": 2, "liked": 2}, "205430": {"suggested": 0, "seen": 0, "liked": 2}, "118338": {"suggested": 0, "seen": 1, "liked": 1}, "177915": {"suggested": 0, "seen": 1, "liked": 1}, "132562": {"suggested": 0, "seen": 1, "liked": 1}, "76279": {"suggested": 1, "seen": 0, "liked": 2}, "158950": {"suggested": 0, "seen": 1, "liked": 1}, "144779": {"suggested": 1, "seen": 1, "liked": 1}}

In [2]:
movieMentions

{'173191': 'House at the End of the Street (2012)',
 '205430': 'A Quiet Place (2018)',
 '118338': 'The Forest  (2016)',
 '177915': 'Uncle Buck',
 '132562': 'The Last House on the Left  (1972)',
 '76279': 'Death at a Funeral  (2007)',
 '158950': 'The Last House on the Left  (2009)',
 '144779': 'Annabelle 2 (2017)'}

In [3]:
seekerPolarity

{'173191': {'suggested': 1, 'seen': 2, 'liked': 2},
 '205430': {'suggested': 0, 'seen': 0, 'liked': 2},
 '118338': {'suggested': 0, 'seen': 1, 'liked': 1},
 '177915': {'suggested': 0, 'seen': 1, 'liked': 1},
 '132562': {'suggested': 0, 'seen': 1, 'liked': 1},
 '76279': {'suggested': 1, 'seen': 0, 'liked': 2},
 '158950': {'suggested': 0, 'seen': 1, 'liked': 1},
 '144779': {'suggested': 1, 'seen': 1, 'liked': 1}}

In [5]:
ssas = [173191]
ori_movie_id = "173191"
int(ori_movie_id) in ssas

True

In [ ]:
defaultdict({'20001': 
                    {'111776': 'Super Troopers (2001)',
                    '91481': 'Beverly Hills Cop (1984)',
                    '151656': 'Police Academy  (1984)',
                    '134643': 'American Pie  (1999)',
                    '192131': 'American Pie ',
                    '124771': '48 Hrs. (1982)',
                    '94688': 'Police Academy 2: Their First Assignment (1985)',
                    '101794': 'Lethal Weapon (1987)'})
            
            
"aspects": [18292] # movieid = 111776

In [ ]:
redial_mv_id = re.findall(r'@(\d+)', message["text"])

여러개면,, 
for mv_id in redial_mv_id:
    mv_id

In [ ]:
# "76012": 29959 (entity_mv_id)


except:
    for k, v in new_redial_movie_id2_entityid.items():
        if v == entity_mv_id:
            try: # 동일 entity id가 여러 key (original redial movie id)에 매핑되었을 수도 있음
                movie_title = movieid2name[v]
            except:
                continue